# 11.02 - Dominando Polars: Rendimiento y Expresividad

**Autor:** Miguel Angel Vazquez Varela  
**Nivel:** Avanzado  
**Tiempo estimado:** 60 min

---

## ¿Por qué Polars?

Polars es una librería de manipulación de datos escrita en **Rust**, diseñada desde cero para aprovechar el paralelismo de las CPUs modernas (SIMD) y la memoria eficiente de Apache Arrow.

### Diferencias clave con Pandas:
- **Multi-threading:** Polars utiliza todos los núcleos de tu CPU automáticamente.
- **Lazy Evaluation:** Optimiza tus consultas antes de ejecutarlas.
- **Sin Index:** Elimina la complejidad y sobrecarga de los índices de Pandas.
- **Tipado Estricto:** Menos errores silenciosos de tipos de datos.

In [1]:
import pandas as pd
import numpy as np
import time
import datetime

try:
    import polars as pl
    POLARS_AVAILABLE = True
    print(f"Polars version: {pl.__version__}")
except ImportError:
    POLARS_AVAILABLE = False
    print("Polars no instalado. Instalar con: pip install polars")

Polars version: 1.29.0


---

## 1. Benchmarking: El Salto de Rendimiento

Generamos un dataset de **10 millones de filas** para ver la diferencia real.

In [2]:
size = 10_000_000
data = {
    'id': np.arange(size),
    'category': np.random.choice(['A', 'B', 'C', 'D'], size=size),
    'value': np.random.randn(size),
    'date': pd.date_range('2020-01-01', periods=size, freq='s')
}

df_pd = pd.DataFrame(data)
if POLARS_AVAILABLE:
    df_pl = pl.from_pandas(df_pd)

In [3]:
# Tarea: Calcular la media por categoría y el valor máximo por año

print("--- Benchmark: Agregaciones Complejas ---")

# Pandas
start = time.time()
res_pd = df_pd.assign(year=df_pd['date'].dt.year).groupby(['category', 'year'])['value'].agg(['mean', 'max'])
print(f"Pandas: {time.time() - start:.4f}s")

if POLARS_AVAILABLE:
    # Polars
    start = time.time()
    res_pl = (
        df_pl.with_columns(pl.col('date').dt.year().alias('year'))
        .group_by(['category', 'year'])
        .agg([
            pl.col('value').mean().alias('mean'),
            pl.col('value').max().alias('max')
        ])
    )
    print(f"Polars: {time.time() - start:.4f}s")

--- Benchmark: Agregaciones Complejas ---


Pandas: 4.4887s


Polars: 0.8619s


---

## 2. El Poder de las Expresiones

En Polars, no modificas columnas una por una; describes **qué** quieres hacer.

In [4]:
if POLARS_AVAILABLE:
    # Podemos aplicar múltiples lógicas en un solo paso
    transformed = df_pl.select([
        pl.col('id'),
        pl.col('category').str.to_lowercase(),
        (pl.col('value') * 10).round(2).alias('v_x10'),
        pl.col('date').dt.weekday().alias('day_of_week'),
        # Condiciones lógicas complejas
        pl.when(pl.col('value') > 2).then(pl.lit('High')).otherwise(pl.lit('Normal')).alias('status')
    ]).head(5)
    
    display(transformed)

id,category,v_x10,day_of_week,status
i32,str,f64,i8,str
0,"""b""",1.66,3,"""Normal"""
1,"""c""",11.97,3,"""Normal"""
2,"""a""",1.5,3,"""Normal"""
3,"""b""",6.51,3,"""Normal"""
4,"""c""",14.31,3,"""Normal"""


---

## 3. Window Functions: Análisis de Rankings y Over

Una de las joyas de Polars es la capacidad de usar la expresión `.over()`, similar a SQL.

In [5]:
if POLARS_AVAILABLE:
    # Calcular la diferencia respecto a la media de su propia categoría sin necesidad de Join
    df_window = df_pl.head(100).with_columns([
        pl.col('value').mean().over('category').alias('category_avg'),
        (pl.col('value') - pl.col('value').mean().over('category')).alias('diff_from_avg'),
        pl.col('id').count().over('category').alias('group_size')
    ])
    
    display(df_window.head(5))

id,category,value,date,category_avg,diff_from_avg,group_size
i32,str,f64,datetime[ns],f64,f64,u32
0,"""B""",0.165726,2020-01-01 00:00:00,-0.103381,0.269106,21
1,"""C""",1.196667,2020-01-01 00:00:01,0.251299,0.945368,29
2,"""A""",0.149586,2020-01-01 00:00:02,-0.146691,0.296277,22
3,"""B""",0.651327,2020-01-01 00:00:03,-0.103381,0.754708,21
4,"""C""",1.431053,2020-01-01 00:00:04,0.251299,1.179754,29


---

## 4. Lazy API & Optimization

El modo *Lazy* permite a Polars reordenar filtros para leer menos datos de disco o memoria.

In [6]:
if POLARS_AVAILABLE:
    # Creamos un plan lógico
    q = (
        df_pl.lazy()
        .filter(pl.col('category') == 'A')
        .with_columns((pl.col('value') ** 2).alias('val_sq'))
        .group_by('category')
        .agg(pl.col('val_sq').sum())
    )
    
    print("--- Plan Optimizado ---")
    print(q.explain())
    
    # Solo se ejecuta aquí
    result = q.collect()

--- Plan Optimizado ---
AGGREGATE
  [col("val_sq").sum()] BY [col("category")]
  FROM
   WITH_COLUMNS:
   [col("value").pow([dyn int: 2]).alias("val_sq")] 
    FILTER [(col("category")) == ("A")]
    FROM
      DF ["id", "category", "value", "date"]; PROJECT["category", "value"] 2/4 COLUMNS


---

## 5. Streaming: Datos más grandes que la RAM

Polars puede procesar archivos gigantes que no caben en memoria procesándolos en 'chunks'.

In [7]:
if POLARS_AVAILABLE:
    # Ejemplo de estructura de comando para streaming
    # (Requiere lectura de archivos tipo Parquet o CSV)
    print("Estructura streaming:")
    print("df = pl.scan_parquet('big_file.parquet').filter(...).collect(streaming=True)")

Estructura streaming:
df = pl.scan_parquet('big_file.parquet').filter(...).collect(streaming=True)


---

## Resumen Técnico

| Operación | Syntax Polars | Notas |
|-----------|---------------|-------|
| Crear Columna | `.with_columns()` | Evita mutaciones in-place |
| Filtrar | `.filter()` | Mucho más legible que `df[df[...]]` |
| Seleccionar | `.select()` | Muy eficiente para proyecciones |
| Agregados | `.agg()` | Permite listas de expresiones |
| Window | `.over()` | Extremadamente potente |

**Consejo:** Si tu dataset tiene más de 500MB, empieza a usar Polars. Tu productividad y tu CPU te lo agradecerán.